# This notebook uses a transformation that is no longer supported - bin integer

In [1]:
import pandas as pd
from pandas.api.types import infer_dtype

import numpy as np

import csv
import pprint
import pickle
import itertools
from itertools import combinations
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None

MAX_CATEGORIES = 10
continuous_columns = []
categorical_columns = []

In [2]:
DATA_DIR_NAME = '/Users/karenblakemore/merck/data/'
DATA_SET_NAME = 'ufo_sightings'
# DATA_SET_NAME = 'ufo_sightings_v4_20180123_215640_449-2018.04.06.19.01.45'
# DATA_SET_NAME = 'hospital_readmission'
# DATA_SET_NAME = 'CauseOfDeath_Demographic_2015-2016'
# DATA_SET_NAME = 'Motor_Vehicle_Crashes_-_Vehicle_Information__Three_Year_Window'

with open(DATA_DIR_NAME + DATA_SET_NAME + '.pkl', 'rb') as f:
    feature_sets = pickle.load(f)

# Transformations used: type inference, date/time decomposition, zero variance, perfect positive correlation
pdf = feature_sets[7]['df']
column_types = feature_sets[7]['column_types']
print(feature_sets[7]['type'])

pprint.pprint(column_types)

for col in pdf:
    print(pdf[col].describe())
    
display(pdf.columns.values)
display(pdf.head())

lowercase
{'city': 'string',
 'comments': 'string',
 'country': 'string',
 'date posted-day': 'categorical',
 'date posted-month': 'categorical',
 'date posted-year': 'categorical',
 'datetime-day': 'categorical',
 'datetime-hour': 'categorical',
 'datetime-minute': 'categorical',
 'datetime-month': 'categorical',
 'datetime-year': 'categorical',
 'duration hours/min': 'string',
 'duration seconds': 'categorical',
 'latitude': 'categorical',
 'longitude': 'categorical',
 'shape': 'string',
 'state': 'string'}
count       80332
unique      19900
top       seattle
freq          525
Name: city, dtype: object
count                 80317
unique                79828
top       lights in the sky
freq                     17
Name: comments, dtype: object
count     70662
unique        5
top          us
freq      65114
Name: country, dtype: object
count         80332
unique         8346
top       5 minutes
freq           4716
Name: duration hours/min, dtype: object
count                         80

array(['city', 'comments', 'country', 'duration hours/min',
       'duration seconds', 'latitude', 'longitude', 'shape', 'state',
       'date posted-year', 'date posted-month', 'date posted-day',
       'datetime-year', 'datetime-month', 'datetime-day', 'datetime-hour',
       'datetime-minute'], dtype=object)

,city,comments,country,duration hours/min,duration seconds,latitude,longitude,shape,state,date posted-year,date posted-month,date posted-day,datetime-year,datetime-month,datetime-day,datetime-hour,datetime-minute
0,san marcos,this event took place in early fall around 194...,us,45 minutes,"(-97835.999, 9783600.001]","(26.031, 41.587]","(-105.638, -70.128]",cylinder,tx,"(2002.8, 2004.4]","(3.2, 4.3]","(25.0, 28.0]","(1938.4, 1949.2]","(9.8, 10.9]","(7.0, 10.0]","(18.4, 20.7]","(29.5, 35.4]"
1,lackland afb,1949 lackland afb&#44 tx. lights racing acros...,NaN,1-2 hrs,"(-97835.999, 9783600.001]","(26.031, 41.587]","(-105.638, -70.128]",light,tx,"(2004.4, 2006.0]","(10.9, 12.0]","(13.0, 16.0]","(1938.4, 1949.2]","(9.8, 10.9]","(7.0, 10.0]","(20.7, 23.0]","(-0.059, 5.9]"
2,chester (uk/england),green/orange circular disc over chester&#44 en...,gb,20 seconds,"(-97835.999, 9783600.001]","(41.587, 57.144]","(-34.618, 0.892]",circle,NaN,"(2007.6, 2009.2]","(0.989, 2.1]","(19.0, 22.0]","(1949.2, 1960.0]","(9.8, 10.9]","(7.0, 10.0]","(16.1, 18.4]","(-0.059, 5.9]"
3,edna,my older brother and twin sister were leaving ...,us,1/2 hour,"(-97835.999, 9783600.001]","(26.031, 41.587]","(-105.638, -70.128]",circle,tx,"(2002.8, 2004.4]","(0.989, 2.1]","(16.0, 19.0]","(1949.2, 1960.0]","(9.8, 10.9]","(7.0, 10.0]","(20.7, 23.0]","(-0.059, 5.9]"
4,kaneohe,as a marine 1st lt. flying an fj4b fighter/att...,us,15 minutes,"(-97835.999, 9783600.001]","(10.475, 26.031]","(-177.013, -141.148]",light,hi,"(2002.8, 2004.4]","(0.989, 2.1]","(19.0, 22.0]","(1949.2, 1960.0]","(9.8, 10.9]","(7.0, 10.0]","(18.4, 20.7]","(-0.059, 5.9]"


## Convert all variables to Strings
This also converts NaN and NaT to the string 'nan'.

In [3]:
for col in pdf:
    pdf[col] = pdf[col].astype('str')

display(pdf.head())

,city,comments,country,duration hours/min,duration seconds,latitude,longitude,shape,state,date posted-year,date posted-month,date posted-day,datetime-year,datetime-month,datetime-day,datetime-hour,datetime-minute
0,san marcos,this event took place in early fall around 194...,us,45 minutes,"(-97835.999, 9783600.001]","(26.031, 41.587]","(-105.638, -70.128]",cylinder,tx,"(2002.8, 2004.4]","(3.2, 4.3]","(25.0, 28.0]","(1938.4, 1949.2]","(9.8, 10.9]","(7.0, 10.0]","(18.4, 20.7]","(29.5, 35.4]"
1,lackland afb,1949 lackland afb&#44 tx. lights racing acros...,nan,1-2 hrs,"(-97835.999, 9783600.001]","(26.031, 41.587]","(-105.638, -70.128]",light,tx,"(2004.4, 2006.0]","(10.9, 12.0]","(13.0, 16.0]","(1938.4, 1949.2]","(9.8, 10.9]","(7.0, 10.0]","(20.7, 23.0]","(-0.059, 5.9]"
2,chester (uk/england),green/orange circular disc over chester&#44 en...,gb,20 seconds,"(-97835.999, 9783600.001]","(41.587, 57.144]","(-34.618, 0.892]",circle,nan,"(2007.6, 2009.2]","(0.989, 2.1]","(19.0, 22.0]","(1949.2, 1960.0]","(9.8, 10.9]","(7.0, 10.0]","(16.1, 18.4]","(-0.059, 5.9]"
3,edna,my older brother and twin sister were leaving ...,us,1/2 hour,"(-97835.999, 9783600.001]","(26.031, 41.587]","(-105.638, -70.128]",circle,tx,"(2002.8, 2004.4]","(0.989, 2.1]","(16.0, 19.0]","(1949.2, 1960.0]","(9.8, 10.9]","(7.0, 10.0]","(20.7, 23.0]","(-0.059, 5.9]"
4,kaneohe,as a marine 1st lt. flying an fj4b fighter/att...,us,15 minutes,"(-97835.999, 9783600.001]","(10.475, 26.031]","(-177.013, -141.148]",light,hi,"(2002.8, 2004.4]","(0.989, 2.1]","(19.0, 22.0]","(1949.2, 1960.0]","(9.8, 10.9]","(7.0, 10.0]","(18.4, 20.7]","(-0.059, 5.9]"


## Compute conditional probabilities between pairs of columns

In [ ]:
M, N = pdf.shape
cond_probs = np.empty([N, M, N])  # cond_probs[a, i, b) = P(Ai|Bi), for Ai = pdf[A][i] and Bi = pdf[B][i]

# Calculating P(Ai/Bi) = P(Ai and Bi)/P(Bi)
for b in range(N):
    col_b = pdf.columns[b]
    prob_b = pdf.groupby(col_b).size().div(M)  # P(B)

    for a in range(N):
        col_a = pdf.columns[a]

        groups = pdf.groupby([col_a, col_b]).size().div(M).div(prob_b, axis=0, level=1) # P(A|B) - grouped by A
        
        for i in range(M):
            Ai, Bi = pdf.iloc[i,[a,b]]
            cond_probs[a, i, b] = groups[Ai,Bi] 
            
    print(col_a, col_b)        

datetime-minute city
datetime-minute comments
datetime-minute country
datetime-minute duration hours/min
datetime-minute duration seconds
